In [1]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 16.4 MB/s eta 0:00:00
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 24

In [2]:
! sudo apt install nvidia-utils-515

^C


In [3]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
from transformers import pipeline, set_seed

import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")


In [ ]:
device="cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
model_ckpt = "google/pegasus-cnn_dailymail"


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dataset_samsum=load_dataset("samsum")

In [ ]:
dataset_samsum

In [ ]:
split_lengths=[len(dataset_samsum[split])for split in dataset_samsum]
split_lengths

In [ ]:
print(f"features: {dataset_samsum['train'].column_names}")

In [ ]:
 print("\nDialogue")
 print(dataset_samsum["test"][1]["dialogue"])
 print("\nSummary")
 print(dataset_samsum["test"][1]["summary"])

In [ ]:
pipe=pipeline('summarization',model=model_ckpt)

In [ ]:
dialogue=dataset_samsum['test'][0]['dialogue']

In [ ]:
pipeout=pipe(dialogue)
pipeout

In [ ]:
print(pipeout[0]['summary_text'].replace(" .<n>", ".\n"))

we will pass the data batch wise. it will look through all the data and give us final accuracy

In [ ]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

In [ ]:
def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score

In [ ]:
rouge_metric=load_metric('rouge')

In [ ]:
score = calculate_metric_on_test_ds(dataset_samsum['test'], rouge_metric, model_pegasus, tokenizer, column_text = 'dialogue', column_summary='summary', batch_size=8)


In [ ]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = ['pegasus'])

In [ ]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

In [ ]:
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched = True)


In [ ]:
dataset_samsum_pt['train'][0]

In [ ]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)


In [ ]:
pip install accelerate -U


In [ ]:
!pip install transformers[torch]




In [ ]:
!pip install transformers[integrations]

In [ ]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

In [ ]:
!pip install -q -U bitsandbytes #for reducing floating bits size of training parameters
# !pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install transformers==4.31
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install evaluate
!pip install -qqq trl==0.7.1 #helpful library for fine tuning llm for both supervised and rlhf fine tuning

In [ ]:
output_dir='pegasus-samsum'

In [ ]:


trainer_args = TrainingArguments(
    output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

In [ ]:
trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"],
                  eval_dataset=dataset_samsum_pt["validation"])


In [ ]:
trainer.train()


In [ ]:
model_pegasus.save_pretrained("pegasus-samsum-model")

In [ ]:
tokenizer.save_pretrained("tokenizer")

In [ ]:
sample_text = dataset_samsum["test"][0]["dialogue"]

reference = dataset_samsum["test"][0]["summary"]


gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}

pipe = pipeline("summarization", model="pegasus-samsum-model",tokenizer=tokenizer)

In [ ]:
print("Dialogue:")
print(sample_text)


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

In [ ]:
sample_text = """Summarize:A United States plan to build a temporary port off Gaza’s coast to step up the delivery of humanitarian aid has been criticised as an attempt to divert attention from hundreds of thousands of starving Palestinians and Israel’s consistent blocking of assistance to the enclave.

US President Joe Biden said in his State of the Union speech on Thursday that he was directing the US military to lead an emergency mission to set up a pier off Gaza’s Mediterranean coast to receive ships carrying food, water, medicine and temporary shelters."""

In [ ]:
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

In [ ]:
sample_text="""summarize:Trudy: Hey, so I’m having a party at my place next weekend. Do you want to come?

Ruth: Sure! That sounds like fun. Who else is coming?

Trudy: Let’s see. I think it’s going to be Jerome, Talia, Anna, Juan, Celeste, Michelle and possibly Jamie. It’s not really going to be a party, more like a small get-together. I’m cooking dinner, and we can just hang out.

Ruth: What time should I be there?

Trudy: Oh, anytime between 6 and 7 would be fine.

Ruth: Can I bring anything?

Trudy: Oh, don’t worry about it. I have everything covered.

Ruth: Can I at least bring a bottle of wine?

Trudy: Well, I’m not going to say no to wine. I’m sure that would be appreciated.

Ruth: I’ll do that, then. Thanks for inviting me. """

In [ ]:
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])